## Training a neural network with PSO

In [2]:
# Import modules
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import load_iris


# Import PySwarms
import pyswarms as ps

# Some more magic so that the notebook will reload external python modules;
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

In [3]:
data = load_iris()

In [4]:
# Store the features as X and the labels as y
X = data.data
y = data.target

In [5]:
n_inputs = 4
n_hidden = 20
n_classes = 3

num_samples = 150

In [6]:
def logits_function(p):
    """ Calculate roll-back the weights and biases

    Inputs
    ------
    p: np.ndarray
        The dimensions should include an unrolled version of the
        weights and biases.

    Returns
    -------
    numpy.ndarray of logits for layer 2

    """
    # Roll-back the weights and biases
    W1 = p[0:80].reshape((n_inputs,n_hidden))
    b1 = p[80:100].reshape((n_hidden,))
    W2 = p[100:160].reshape((n_hidden,n_classes))
    b2 = p[160:163].reshape((n_classes,))

    # Perform forward propagation
    z1 = X.dot(W1) + b1  # Pre-activation in Layer 1
    a1 = np.tanh(z1)     # Activation in Layer 1
    logits = a1.dot(W2) + b2 # Pre-activation in Layer 2
    return logits          # Logits for Layer 2

In [7]:
# Forward propagation
def forward_prop(params):
    """Forward propagation as objective function

    This computes for the forward propagation of the neural network, as
    well as the loss.

    Inputs
    ------
    params: np.ndarray
        The dimensions should include an unrolled version of the
        weights and biases.

    Returns
    -------
    float
        The computed negative log-likelihood loss given the parameters
    """

    logits = logits_function(params)

    # Compute for the softmax of the logits
    exp_scores = np.exp(logits)
    probs = exp_scores / np.sum(exp_scores, axis=1, keepdims=True)

    # Compute for the negative log likelihood

    corect_logprobs = -np.log(probs[range(num_samples), y])
    loss = np.sum(corect_logprobs) / num_samples

    return loss

In [8]:
def f(x):
    """Higher-level method to do forward_prop in the
    whole swarm.

    Inputs
    ------
    x: numpy.ndarray of shape (n_particles, dimensions)
        The swarm that will perform the search

    Returns
    -------
    numpy.ndarray of shape (n_particles, )
        The computed loss for each particle
    """
    n_particles = x.shape[0]
    j = [forward_prop(x[i]) for i in range(n_particles)]
    return np.array(j)

In [9]:
%%time
# Initialize swarm
options = {'c1': 0.5, 'c2': 0.3, 'w':0.9}

# Call instance of PSO
dimensions = (n_inputs * n_hidden) + (n_hidden * n_classes) + n_hidden + n_classes
optimizer = ps.single.GlobalBestPSO(n_particles=100, dimensions=dimensions, options=options)

# Perform optimization
cost, pos = optimizer.optimize(f, iters=1000)

2024-09-12 10:31:07,992 - pyswarms.single.global_best - INFO - Optimize for 1000 iters with {'c1': 0.5, 'c2': 0.3, 'w': 0.9}
pyswarms.single.global_best: 100%|██████████|1000/1000, best_cost=0.0295
2024-09-12 10:31:28,647 - pyswarms.single.global_best - INFO - Optimization finished | best cost: 0.02954681342975925, best pos: [ 3.27357184e-01  1.08332674e-03  1.81678910e-02  8.25103618e-01
 -9.33073145e-01  5.42225049e-01  9.39830570e-02 -2.88061415e+00
 -1.36690627e+00 -9.60005625e-01  7.22770679e-01 -4.76620419e-01
  9.00193680e-01  6.81862026e-02  8.36601761e-01  4.06368011e-01
  3.71606435e-01  1.20569435e+00  4.12212604e+00  5.18986566e-01
  5.88584807e-01 -5.16414432e+00  7.26243624e-01  1.70649712e+00
  5.86716439e-01  7.40616701e-02 -1.15590054e+00 -3.44399757e-01
  8.38747308e-01  1.01894544e+00 -3.06795063e-01  3.42097228e-01
  1.34279778e-01  8.45106645e-01 -5.73937175e-01 -2.28537350e-01
  3.58674927e-02 -5.99827056e-01  1.83647510e-01  1.29139123e+00
 -1.00612133e+00 -7.891

CPU times: total: 8.19 s
Wall time: 20.7 s


In [10]:
def predict(pos):
    """
    Use the trained weights to perform class predictions.

    Inputs
    ------
    pos: numpy.ndarray
        Position matrix found by the swarm. Will be rolled
        into weights and biases.
    """
    logits = logits_function(pos)
    y_pred = np.argmax(logits, axis=1)
    return y_pred

In [11]:
(predict(pos) == y).mean()

0.9866666666666667